In [43]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [37]:
clients = pd.read_csv("../data/raw/clients.csv")
clients.head()

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,000048b7a6,2018-12-15 13:33:11,NaN,68,F
3,000073194a,2017-05-23 12:56:14,2017-11-24 11:18:01,60,F
4,00007c7133,2017-05-22 16:17:08,2018-12-31 17:17:33,67,U


In [39]:
clients.shape

(400162, 5)

In [44]:
purchases_client_ids = pd.read_csv("../data/raw/purchases.csv", usecols=["client_id"])
purchases_client_ids["count"] = 1
purchases_client_ids.head()

,client_id,count
0,000012768d,1
1,000012768d,1
2,000012768d,1
3,000012768d,1
4,000012768d,1


In [45]:
purchases_client_ids = purchases_client_ids.groupby("client_id").sum()
purchases_client_ids.head()

,count
client_id,
000012768d,52
000036f903,162
000048b7a6,56
000073194a,82
00007c7133,83


In [48]:
frames = []
skip = 0
i = 0
for c_id, c in purchases_client_ids.iterrows():
    i += 1
    if i % 5000 == 0:
        print(f"{i} of {purchases_client_ids.shape[0]}")
    frame = pd.read_csv("purchases_level_2.csv", skiprows=[i for i in range(1, skip + 1)], nrows=c[0])
    data = (
        frame.groupby("client_id")[
            [x for x in frame.columns if x.startswith("level_") or x.startswith("segment_id_")]
        ].sum()
        .reset_index()
        .groupby("client_id")
        .agg([sum, np.mean, np.std])
    )
    data.columns = ['_'.join(col) for col in data.columns]
    frames.append(data)
    skip += c[0]
df = pd.concat(frames, ignore_index=False)
df.to_csv("../data/processed/level_2.csv", index=True)

KeyboardInterrupt: 

In [ ]:
frames = []
skip = 0
for c_id, c in purchases_client_ids.iterrows():
    frame = pd.read_csv("purchases_segment.csv", skiprows=[i for i in range(1, skip + 1)], nrows=c[0])
    data = (
        frame.groupby("client_id")[
            [x for x in frame.columns if x.startswith("level_") or x.startswith("segment_id_")]
        ].sum()
        .reset_index()
        .groupby("client_id")
        .agg([sum, np.mean, np.std])
    )
    data.columns = ['_'.join(col) for col in data.columns]
    frames.append(data)
    skip += c[0]
df = pd.concat(frames, ignore_index=False)
df.to_csv("segment.csv", index=True)

In [25]:
purchases_datetime = pd.read_csv(
    "../data/raw/purchases.csv", usecols=["client_id", "transaction_id", "transaction_datetime"], parse_dates=["transaction_datetime"]
)
purchases_datetime.head()

,client_id,transaction_id,transaction_datetime
0,000012768d,7e3e2e3984,2018-12-01 07:12:45
1,000012768d,7e3e2e3984,2018-12-01 07:12:45
2,000012768d,7e3e2e3984,2018-12-01 07:12:45
3,000012768d,7e3e2e3984,2018-12-01 07:12:45
4,000012768d,7e3e2e3984,2018-12-01 07:12:45


In [26]:
purchases_datetime = purchases_datetime.sort_values(by=["client_id", "transaction_datetime"]).reset_index(drop=True)

In [27]:
purchases_datetime.head(20)

,client_id,transaction_id,transaction_datetime
0,000012768d,7e3e2e3984,2018-12-01 07:12:45
1,000012768d,7e3e2e3984,2018-12-01 07:12:45
2,000012768d,7e3e2e3984,2018-12-01 07:12:45
3,000012768d,7e3e2e3984,2018-12-01 07:12:45
4,000012768d,7e3e2e3984,2018-12-01 07:12:45
5,000012768d,7e3e2e3984,2018-12-01 07:12:45
6,000012768d,7e3e2e3984,2018-12-01 07:12:45
7,000012768d,7e3e2e3984,2018-12-01 07:12:45
8,000012768d,7e3e2e3984,2018-12-01 07:12:45
9,000012768d,7e3e2e3984,2018-12-01 07:12:45


In [28]:
grouped_data = purchases_datetime.groupby(["client_id", "transaction_id"], sort=False, as_index=False)["transaction_datetime"].first()
grouped_data.head(15)

,client_id,transaction_id,transaction_datetime
0,000012768d,7e3e2e3984,2018-12-01 07:12:45
1,000012768d,c1ca85d462,2018-12-16 08:56:01
2,000012768d,6a0e96d0bc,2019-03-08 10:12:03
3,000012768d,b34f23306e,2019-03-14 15:01:47
4,000036f903,12b218b054,2018-11-28 10:48:36
5,000036f903,e90f8f4e1f,2018-12-03 10:24:30
6,000036f903,462ba5a47d,2018-12-06 10:42:27
7,000036f903,96f11b0933,2018-12-10 10:43:28
8,000036f903,ff8c303a03,2018-12-12 11:13:37
9,000036f903,1af98450b3,2018-12-16 10:13:17


In [31]:
grouped_data["lag"] = grouped_data.groupby(["client_id"], sort=False)["transaction_datetime"].shift(1)
grouped_data.head(15)

,client_id,transaction_id,transaction_datetime,lag
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,NaT
1,000012768d,c1ca85d462,2018-12-16 08:56:01,2018-12-01 07:12:45
2,000012768d,6a0e96d0bc,2019-03-08 10:12:03,2018-12-16 08:56:01
3,000012768d,b34f23306e,2019-03-14 15:01:47,2019-03-08 10:12:03
4,000036f903,12b218b054,2018-11-28 10:48:36,NaT
5,000036f903,e90f8f4e1f,2018-12-03 10:24:30,2018-11-28 10:48:36
6,000036f903,462ba5a47d,2018-12-06 10:42:27,2018-12-03 10:24:30
7,000036f903,96f11b0933,2018-12-10 10:43:28,2018-12-06 10:42:27
8,000036f903,ff8c303a03,2018-12-12 11:13:37,2018-12-10 10:43:28
9,000036f903,1af98450b3,2018-12-16 10:13:17,2018-12-12 11:13:37


In [35]:
grouped_data["diff"] = (grouped_data["transaction_datetime"] - grouped_data["lag"]).dt.total_seconds()
grouped_data.head(15)

,client_id,transaction_id,transaction_datetime,lag,diff
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,NaT,NaN
1,000012768d,c1ca85d462,2018-12-16 08:56:01,2018-12-01 07:12:45,1302196.0
2,000012768d,6a0e96d0bc,2019-03-08 10:12:03,2018-12-16 08:56:01,7089362.0
3,000012768d,b34f23306e,2019-03-14 15:01:47,2019-03-08 10:12:03,535784.0
4,000036f903,12b218b054,2018-11-28 10:48:36,NaT,NaN
5,000036f903,e90f8f4e1f,2018-12-03 10:24:30,2018-11-28 10:48:36,430554.0
6,000036f903,462ba5a47d,2018-12-06 10:42:27,2018-12-03 10:24:30,260277.0
7,000036f903,96f11b0933,2018-12-10 10:43:28,2018-12-06 10:42:27,345661.0
8,000036f903,ff8c303a03,2018-12-12 11:13:37,2018-12-10 10:43:28,174609.0
9,000036f903,1af98450b3,2018-12-16 10:13:17,2018-12-12 11:13:37,341980.0


In [41]:
grouped_data\
.groupby("client_id")["diff"]\
.agg([np.mean, np.std, np.sum])\
.rename(columns=lambda x: "transactions_freq_" + x)\
.to_csv("../data/processed/frequency.csv", index=True)